In [4]:
# Import necessary libraries for multiple models
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Define a dictionary of models to evaluate
models_dict = {
    "Random Forest": RandomForestRegressor(random_state=42),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Linear Regression": LinearRegression(),
    "XGBoost": XGBRegressor(random_state=42)
}

# Train and evaluate each model
results = {}

for model_name, model in models_dict.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate the Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)
    
    # Store the results
    results[model_name] = mae

# Display the results for comparison
results


TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'